Run detection using trained yolov3 model on our custom dataset. 

In [ ]:
import cv2
import numpy as np
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#give google drive folder alias so we don't have to worry about space in file path
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive/capstone_2

Archive.zip		 bdd_train.zip	     snapshots_training_10k
backup			 classes.names	     tensorboard
backup1			 config.ini	     yolo_bdd_custom_8000.weights
bdd04-detected.mp4	 darknet53.conv.74   yolo_bdd_custom.cfg
bdd04.mp4		 detected5.mp4	     yolo_bdd_custom_last.weights
bdd04-yolo-detected.mp4  docs		     yolo.h5
bdd05-detected.mp4	 nola.jpg	     yolov3.weights
bdd05.mp4		 resnet50_csv_20.h5  yolo-video.ipynb
bdd05-yolo-detected.mp4  snapshots_1000


In [ ]:
def detect_video(video_path, classes_path, config_path, weights_path, save_path):
  video = cv2.VideoCapture(video_path)

  writer = None
  h,w = None, None

  with open(classes_path) as f:
    labels = [line.strip() for line in f]
  
  model = cv2.dnn.readNetFromDarknet(config_path, weights_path)

  layers_names_all = model.getLayerNames()
  layers_names_output = [layers_names_all[i[0] -1] for i in model.getUnconnectedOutLayers()]

  probability_minimum = 0.5
  threshold = 0.3
  colors = colors = np.array([[255, 0, 0], [0,255,0], [0,0,255], [128, 0, 128], [128, 0, 0], [0,255,255], [255, 255, 0], [11, 134, 184], [0,0,0], [192, 192, 192]])

  f = 0
  t = 0

  while True:
    ret, frame = video.read()
    
    if not ret:
        break
    
    if w is None or h is None:
        h,w = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), swapRB=True, crop=False)
    
    
    model.setInput(blob)
    start = time.time()
    output_from_model = model.forward(layers_names_output)
    end = time.time()
    
    f+=1
    t+= end-start
    
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))
    
    
    bounding_boxes=[]
    confidences=[]
    class_numbers=[]
    
    for result in output_from_model:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current=np.argmax(scores)
            confidence_current=scores[class_current]
            
            
            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w,h,w,h])
                
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))
                
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
     
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    
    if len(results) > 0:
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            color_box_current = colors[class_numbers[i]].tolist()
            
            cv2.rectangle(frame, (x_min, y_min), (x_min + box_width, y_min + box_height), color_box_current, 2)
            
            text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])], confidences[i])
            
            cv2.putText(frame, text_box_current, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
            
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
        
        writer = cv2.VideoWriter(save_path, fourcc, 30, (frame.shape[1], frame.shape[0]), True)
        
    writer.write(frame)

  print('Total number of frames', f)
  print('Total amount of time {:.5f} seconds'.format(t))
  print('FPS: ', round((f/t), 1))

  video.release()
  writer.release()

In [ ]:
detect_video('/mydrive/capstone_2/bdd05.mp4', '/mydrive/capstone_2/classes.names', '/mydrive/capstone_2/yolo_bdd_custom.cfg', '/mydrive/capstone_2/yolo_bdd_custom_8000.weights', '/mydrive/capstone_2/test-colors.mp4')

Frame number 1 took 1.54268 seconds
Frame number 2 took 0.90093 seconds
Frame number 3 took 0.92341 seconds
Frame number 4 took 0.90456 seconds
Frame number 5 took 0.89147 seconds
Frame number 6 took 0.91495 seconds
Frame number 7 took 0.89288 seconds
Frame number 8 took 0.89974 seconds
Frame number 9 took 0.93030 seconds
Frame number 10 took 0.92312 seconds
Frame number 11 took 0.90166 seconds
Frame number 12 took 0.91665 seconds
Frame number 13 took 0.91459 seconds
Frame number 14 took 0.90005 seconds
Frame number 15 took 0.90121 seconds
Frame number 16 took 0.91139 seconds
Frame number 17 took 0.90057 seconds
Frame number 18 took 0.91857 seconds
Frame number 19 took 0.90991 seconds
Frame number 20 took 0.90995 seconds
Frame number 21 took 0.89976 seconds
Frame number 22 took 0.90570 seconds
Frame number 23 took 0.89729 seconds
Frame number 24 took 0.91199 seconds
Frame number 25 took 0.90297 seconds
Frame number 26 took 0.87519 seconds
Frame number 27 took 0.90016 seconds
Frame numb